In [1]:
!pip3 -q install ccxt pandas ta

指标说明
| 指标名称              | 说明                           |
| ----------------- | ---------------------------- |
| `rsi`             | RSI(6) 相对强弱指标                |
| `bollinger_*`     | 布林带 上轨/中轨/下轨，2倍标准差，window=20 |
| `volume_ratio_2`  | 当前成交量 / 上一根K线成交量             |
| `volume_ratio_1`  | 上一根K线成交量 / 上两根K线成交量          |
| `hour_avg_volume` | 过去60根K线成交量均值（近一小时均量）         |
| `volume_ratio`    | 当前成交量 / 一小时平均成交量             |


In [6]:
import ccxt
import pandas as pd
import ta  # pip install ta

# 初始化交易所
exchange = ccxt.poloniex()

# 配置参数
symbol = 'BTC/USDT'
timeframe = '15m'
limit = 100

# 获取 OHLCV 数据
ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)

# 转换为 DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# 打印原始数据
print(df[['timestamp', 'close', 'volume']])

# 计算 RSI(6)
df['rsi'] = ta.momentum.rsi(df['close'], window=6)

# 计算 布林带（2倍标准差）
bollinger = ta.volatility.BollingerBands(close=df['close'], window=20, window_dev=2)
df['bollinger_upper'] = bollinger.bollinger_hband()
df['bollinger_mid'] = bollinger.bollinger_mavg()
df['bollinger_lower'] = bollinger.bollinger_lband()

# 增加你指定的指标
df['volume_ratio_1'] = df['volume'] / df['volume'].shift(1)
df['volume_ratio_2'] = df['volume'].shift(1) / df['volume'].shift(2)
df['hour_avg_volume'] = df['volume'].rolling(window=60).mean()
df['volume_ratio'] = df['volume'] / df['hour_avg_volume']

# 打印完整结果，方便调试
df[['timestamp', 'close', 'volume', 'rsi', 'bollinger_upper', 'bollinger_mid', 'bollinger_lower', 
          'volume_ratio_2', 'volume_ratio_1', 'hour_avg_volume', 'volume_ratio']]


             timestamp      close     volume
0  2025-06-10 02:30:00  109582.99  12.626842
1  2025-06-10 02:45:00  109605.44  31.436455
2  2025-06-10 03:00:00  109376.99  29.808595
3  2025-06-10 03:15:00  109405.55  21.467041
4  2025-06-10 03:30:00  109369.77  22.703512
..                 ...        ...        ...
95 2025-06-11 02:15:00  109509.22   9.839909
96 2025-06-11 02:30:00  109596.28  17.713171
97 2025-06-11 02:45:00  109593.36  16.327695
98 2025-06-11 03:00:00  109671.84   8.761059
99 2025-06-11 03:15:00  109765.66  19.184518

[100 rows x 3 columns]


,timestamp,close,volume,rsi,bollinger_upper,bollinger_mid,bollinger_lower,volume_ratio_2,volume_ratio_1,hour_avg_volume,volume_ratio
0,2025-06-10 02:30:00,109582.99,12.626842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-06-10 02:45:00,109605.44,31.436455,NaN,NaN,NaN,NaN,NaN,2.489653,NaN,NaN
2,2025-06-10 03:00:00,109376.99,29.808595,NaN,NaN,NaN,NaN,2.489653,0.948217,NaN,NaN
3,2025-06-10 03:15:00,109405.55,21.467041,NaN,NaN,NaN,NaN,0.948217,0.720163,NaN,NaN
4,2025-06-10 03:30:00,109369.77,22.703512,NaN,NaN,NaN,NaN,0.720163,1.057599,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,2025-06-11 02:15:00,109509.22,9.839909,33.396519,110112.144353,109761.4590,109410.773647,0.429758,0.865867,15.174776,0.648439
96,2025-06-11 02:30:00,109596.28,17.713171,42.722193,110112.304475,109761.2730,109410.241525,0.865867,1.800136,15.195607,1.165677
97,2025-06-11 02:45:00,109593.36,16.327695,42.482783,110112.593975,109760.9410,109409.288025,1.800136,0.921783,15.264051,1.069683
98,2025-06-11 03:00:00,109671.84,8.761059,51.286995,110093.223265,109748.1695,109403.115735,0.921783,0.536577,15.237792,0.574956
